In [6]:
import sys

lib_path = '/work-hmcomm/project/nedo2020_yokogawa/Git/yokogawa_knikaido/fastd-learning/src'
sys.path.insert(0, lib_path)

import numpy as np
import numpy # Since some code's been cut and pasted from elsewhere we need this too 
import os
import matplotlib.pyplot as plt
import glob
import pickle
import scipy.fftpack as spfft
from scipy import signal
import librosa
import librosa.display
from tqdm import tqdm
import pywt

import pandas as pd

In [7]:
from util.util import parse_audio_from_wav
from feature.feature_mfcc import FeatureMfcc

In [8]:
savepath = '/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/feature/extracted_on_2020-06-01_scalogram/'

fs = 96000

In [9]:
wav_list = sorted(glob.glob('/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/*_1_*.wav'))
wav_list.extend(sorted(glob.glob('/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/*_5_*.wav')))
wav_list.extend(sorted(glob.glob('/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/*_6_*.wav')))
wav_list.extend(sorted(glob.glob('/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/*_2_*.wav')))
wav_list.extend(sorted(glob.glob('/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/*_3_*.wav')))
wav_list.extend(sorted(glob.glob('/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/*_4_*.wav')))
# wav_list.pop(5)
wav_list

['/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_1_ZOOM-H06_Exp02.wav',
 '/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_1_ZOOM-H06_Exp03.wav',
 '/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_1_ZOOM-H06_Exp04.wav',
 '/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_1_ZOOM-H06_Exp05.wav',
 '/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_1_ZOOM-H06_Exp06.wav',
 '/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_1_ZOOM-H06_Exp07.wav',
 '/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_1_ZOOM-H06_Exp08.wav',
 '/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_1_ZOOM-H06_Exp09.wav',
 '/work-hmcomm/project/nedo2020_

In [10]:

for j in tqdm(np.arange(1, 7)):
    if(j<=4):
        widths = np.exp(np.arange(j, j+1, 0.15))
    else:
        widths = np.exp(np.arange(j, j+1, 0.1))
    print(widths)
widths = np.exp(np.arange(1, 7, 0.1))
widths

100%|██████████| 6/6 [00:00<00:00, 1936.28it/s]

[2.71828183 3.15819291 3.66929667 4.26311452 4.95303242 5.75460268
 6.68589444]
[ 7.3890561   8.5848584   9.97418245 11.58834672 13.46373804 15.64263188
 18.17414537]
[20.08553692 23.33606458 27.11263892 31.50039231 36.59823444 42.521082
 49.40244911]
[ 54.59815003  63.4340003   73.6997937   85.626944    99.48431564
 115.58428453 134.28977968]
[148.4131591  164.0219073  181.27224188 200.33680997 221.4064162
 244.69193226 270.42640743 298.86740097 330.29955991 365.03746787]
[403.42879349 445.85777008 492.74904109 544.57191013 601.84503787
 665.14163304 735.09518924 812.40582517 897.84729165 992.27471561]


array([  2.71828183,   3.00416602,   3.32011692,   3.66929667,
         4.05519997,   4.48168907,   4.95303242,   5.47394739,
         6.04964746,   6.68589444,   7.3890561 ,   8.16616991,
         9.0250135 ,   9.97418245,  11.02317638,  12.18249396,
        13.46373804,  14.87973172,  16.44464677,  18.17414537,
        20.08553692,  22.19795128,  24.5325302 ,  27.11263892,
        29.96410005,  33.11545196,  36.59823444,  40.44730436,
        44.70118449,  49.40244911,  54.59815003,  60.3402876 ,
        66.68633104,  73.6997937 ,  81.45086866,  90.0171313 ,
        99.48431564, 109.94717245, 121.51041752, 134.28977968,
       148.4131591 , 164.0219073 , 181.27224188, 200.33680997,
       221.4064162 , 244.69193226, 270.42640743, 298.86740097,
       330.29955991, 365.03746787, 403.42879349, 445.85777008,
       492.74904109, 544.57191013, 601.84503787, 665.14163304,
       735.09518924, 812.40582517, 897.84729165, 992.27471561])

In [11]:
start = 0
Fs = fs

widths = np.exp(np.arange(1, 7, 0.1))

for i in range(len(wav_list)):
    
    cwt_ave_sum = []
    wavpath = wav_list[i]
    _, a =   parse_audio_from_wav(wavpath,sample_rate=fs)
    wavdata = a[:,1]
    
    for j in tqdm(np.arange(1, 7)):

        if(j<=5):
            widths = np.exp(np.arange(j, j+1, 0.15))
        else:
            widths = np.exp(np.arange(j, j+1, 0.15))

        cwtmatr, freqs = pywt.cwt(wavdata, widths, 'morl', sampling_period=1/Fs, method='fft')
        # cwtmatr += np.abs(np.min(cwtmatr))
        np.set_printoptions(suppress=True)
        # print(freqs)
        # plt.figure(figsize=(12, 4))
        # plt.imshow(np.abs(cwtmatr),  extent=[0, len(cwtmatr[0]), 0, len(widths)], cmap='PRGn', aspect='auto',
        #             vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max()) 


        win_length=0.4*fs
        hop_length=0.2*fs
        start = 0
        cwtmatr = np.abs(cwtmatr)
        cwt_ave = []

        num_signal = int((len(cwtmatr[0]) - win_length) / hop_length) + 1
        for i in range(num_signal):

            con_cwt = cwtmatr[:,int(start):int(start+win_length)]
            cwt_ave.append(np.log(np.array(np.average(con_cwt ,axis=1))))
            
            start += hop_length

#         cwt_ave = np.array(cwt_ave).T
        cwt_ave_sum.extend(np.array(cwt_ave).T)
    print('**********'+wavpath+'**********')
    cwt_ave_sum = np.array(cwt_ave_sum)
    print(cwt_ave_sum.shape)
    savename = savepath + wavpath[85:-4] + '_scalogram_S200W400_right.npy' 
    np.save(savename, cwt_ave_sum)

100%|██████████| 6/6 [03:42<00:00, 37.12s/it]


**********/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_1_ZOOM-H06_Exp02.wav**********
(42, 1655)


100%|██████████| 6/6 [02:46<00:00, 27.68s/it]


**********/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_1_ZOOM-H06_Exp03.wav**********
(42, 1498)


100%|██████████| 6/6 [02:51<00:00, 28.56s/it]


...

**********/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_4_TAS-D040_Exp09.wav**********
(42, 1498)


100%|██████████| 6/6 [02:42<00:00, 27.12s/it]


**********/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_4_TAS-D040_Exp10.wav**********
(42, 1498)


100%|██████████| 6/6 [23:53<00:00, 238.91s/it]

**********/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/wav_cut/2020-03-06_4_TAS-D040_Exp11.wav**********
(42, 8722)


In [7]:
widths = np.exp(np.arange(1, 7, 0.15))
widths

array([  2.71828183,   3.15819291,   3.66929667,   4.26311452,
         4.95303242,   5.75460268,   6.68589444,   7.76790111,
         9.0250135 ,  10.48556972,  12.18249396,  14.15403865,
        16.44464677,  19.10595373,  22.19795128,  25.79033992,
        29.96410005,  34.81331749,  40.44730436,  46.99306323,
        54.59815003,  63.4340003 ,  73.6997937 ,  85.626944  ,
        99.48431564, 115.58428453, 134.28977968, 156.02246449,
       181.27224188, 210.60829787, 244.69193226, 284.29146582,
       330.29955991, 383.75333906, 445.85777008, 518.01282467,
       601.84503787, 699.24417382, 812.40582517, 943.88090667])

In [27]:
%%time

widths
Fs = fs
cwtmatr, freqs = pywt.cwt(wavdata_, widths, 'morl', sampling_period=1/Fs)
# cwtmatr += np.abs(np.min(cwtmatr))
np.set_printoptions(suppress=True)
print(freqs)
plt.figure(figsize=(12, 4))
# plt.imshow(np.abs(cwtmatr),  extent=[0, len(cwtmatr[0]), 0, len(widths)], cmap='PRGn', aspect='auto',
#             vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max()) 
cwtmatr.shape

[28694.59641137 24697.66801157 21257.47985665 18296.48247132
 15747.92840358 13554.36758913 11666.35229937 10041.32247985
  8642.64635226  7438.79465281  6402.62989266  5510.79461871
  4743.18488477  4082.49706398  3513.8377867   3024.38821087
  2603.11505691  2240.52189303  1928.43506469  1659.81944219
  1428.61983332  1229.62448582  1058.34760295   910.92822369
   784.04318808   674.83222584   580.83347953   499.92800881
   430.29202482   370.35577795   318.7681722    274.36630844
   236.14927014   203.25556044   174.94368212   150.57542263
   129.60146731   111.54901667    96.01112841    82.63754405]
CPU times: user 27min 22s, sys: 19min 19s, total: 46min 42s
Wall time: 13min 30s


(40, 28793856)

<Figure size 864x288 with 0 Axes>

In [28]:

win_length=0.4*fs
hop_length=0.2*fs
start = 0
cwtmatr = np.abs(cwtmatr)
cwt_ave = []

num_signal = int((len(cwtmatr[0]) - win_length) / hop_length) + 1
for i in tqdm(range(num_signal)):

    con_cwt = cwtmatr[:,int(start):int(start+win_length)]
    cwt_ave.append(np.array(np.average(con_cwt ,axis=1)))
#     if(i==0):
#         cwt_ave = np.average(con_cwt ,axis=1)
#     else:
#         cwt_ave = np.vstack([cwt_ave, np.average(con_cwt ,axis=1)])
#     start += hop_length
#     break

plt.figure(figsize=(12, 4))
# plt.imshow(cwt_ave.T,  extent=[0, len(cwt_ave.T[0]), 0, len(widths)], cmap='PRGn', aspect='auto',
#             vmax=abs(cwt_ave.T).max(), vmin=-abs(cwt_ave.T).max()) 
# cwt_ave.T[1]

100%|██████████| 1498/1498 [00:01<00:00, 1168.31it/s]


<Figure size 864x288 with 0 Axes>

<Figure size 864x288 with 0 Axes>

In [13]:
cwt_ave = np.array(cwt_ave).T

array([[5.40299924e+06, 5.40299924e+06, 5.40299924e+06, ...,
        5.40299924e+06, 5.40299924e+06, 5.40299924e+06],
       [8.32306376e+06, 8.32306376e+06, 8.32306376e+06, ...,
        8.32306376e+06, 8.32306376e+06, 8.32306376e+06],
       [1.28420653e+07, 1.28420653e+07, 1.28420653e+07, ...,
        1.28420653e+07, 1.28420653e+07, 1.28420653e+07],
       ...,
       [2.53887318e+08, 2.53887318e+08, 2.53887318e+08, ...,
        2.53887318e+08, 2.53887318e+08, 2.53887318e+08],
       [2.41329505e+08, 2.41329505e+08, 2.41329505e+08, ...,
        2.41329505e+08, 2.41329505e+08, 2.41329505e+08],
       [2.55568819e+08, 2.55568819e+08, 2.55568819e+08, ...,
        2.55568819e+08, 2.55568819e+08, 2.55568819e+08]])

In [8]:
savename = savepath + wav_list[0][85:-4] + '_scalogram_S200W400_left.npy' 
savename

'/work-hmcomm/project/nedo2020_yokogawa/data/PilotPlant_Experiments_2020-0309/feature/extracted_on_2020-06-01_scalogram/2020-03-06_1_ZOOM-H06_Exp08_scalogram_S200W400_lety.npy'

In [10]:
    print('**********'+'**********')

********************
